In [21]:
#uses DotProduct_Systolic - v1.1x
#uses RegisteredMultiplier  - v1.0x
#uses AdderTree - v1.2x

In [22]:
import sys
import math

In [23]:
#N>=2
N = input('Enter N: ')

Enter N: 9


In [24]:
N = int(N)
if(N<2):
    N=2

In [25]:
#MAMCS>=2
MAMCS = input('Enter Multiply-Add Max Component Size (>=2, <= {}, empty for 2): '.format(N-1))

Enter Multiply-Add Max Component Size (>=2, <= 8, empty for 2): 4


In [26]:
#AdderSize>=2
AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enter maximum size allowed for base adders (empty for 2): 10


In [27]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

Enter IN_WIDTH Default Value (empty for 10): 


In [28]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')

Enter INPUT_REG_DEPTH Default Value (empty for 1): 


In [29]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')

Enter MULT_PIPE_DEPTH Default Value (empty for 1): 


In [30]:
if (not MAMCS):
    MAMCS = N-1
else:
    MAMCS = int(MAMCS)
if MAMCS>=(N-1):
    MAMCS = N-1
    NumOfMaxSizeComponents=1
    NonMaxSizeComponentSize=1;
else:
    if(MAMCS<2):
        MAMCS=2
    NumOfMaxSizeComponents = math.floor(N/MAMCS)
    if (N/MAMCS) == NumOfMaxSizeComponents:
        NonMaxSizeComponentSize=0
    else:
        NonMaxSizeComponentSize = N - MAMCS*NumOfMaxSizeComponents
NumOfComponents = NumOfMaxSizeComponents + (NonMaxSizeComponentSize!=0)

if not AdderSize:
    AdderSize=2
else:
    AdderSize = int(AdderSize)
    if AdderSize<2:
        AdderSize=2
    if AdderSize>NumOfComponents:
        AdderSize=NumOfComponents

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [31]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./DotProduct_{}_noHR_C{}_A{}_NR.v".format(N, MAMCS, AdderSize), 'w+')

In [32]:
print("`timescale 1ns / 1ps\n", file=of)
print("module DotProduct_{}_noHR_C{}_A{}_NR".format(N, MAMCS, AdderSize), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log2(N)) #>=1
al=-1+lgn #>=0
if al==0:
    print("output signed [2*IN_WIDTH:0] DP,", file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module DotProduct_9_noHR_C4_A3_NR
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, 
input signed [IN_WIDTH-1:0] B0, B1, B2, B3, B4, B5, B6, B7, B8, 
output outReady,
output signed [(2*IN_WIDTH)+3:0] DP,
output earlyOutReady
);



In [33]:
lgMAMCS=math.ceil(math.log2(MAMCS)) #>=1
palM=lgMAMCS-1 #>=0
print("wire DPOutReady;", file=of)
for i in range (NumOfMaxSizeComponents):
    if palM==0:
        print("wire signed [(2*IN_WIDTH):0] DPpart{};".format(i), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{};".format(palM, i), file=of)
    print("DotProduct_Systolic_NR_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(MAMCS), file=of)
    print("\tDPS{}_{}(clk, reset, enable,".format(MAMCS, i), file=of)
    print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(MAMCS):
        print("A{}, ".format(MAMCS*i+j), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(MAMCS):
        print("B{}, ".format(MAMCS*i+j), end='', file=of)
    print(file=of)
    if i==0:
        print("\tDPOutReady,", file=of)
    else:
        print("\tDPOutReady{}, //not used".format(i), file=of)
    print("\tDPpart{},".format(i), file=of)
    print("\tDPEarlyOutReady{}); //not used\n".format(i), file=of)

wire DPOutReady;
wire signed [(2*IN_WIDTH)+1:0] DPpart0;
DotProduct_Systolic_NR_4#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
	DPS4_0(clk, reset, enable,
	inReady,
	A0, A1, A2, A3, 
	B0, B1, B2, B3, 
	DPOutReady,
	DPpart0,
	DPEarlyOutReady0); //not used

wire signed [(2*IN_WIDTH)+1:0] DPpart1;
DotProduct_Systolic_NR_4#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
	DPS4_1(clk, reset, enable,
	inReady,
	A4, A5, A6, A7, 
	B4, B5, B6, B7, 
	DPOutReady1, //not used
	DPpart1,
	DPEarlyOutReady1); //not used



In [34]:
if NonMaxSizeComponentSize>1:
    LGNonMaxSizeComponentSize=math.ceil(math.log2(NonMaxSizeComponentSize)) #>=1
    palN=LGNonMaxSizeComponentSize-1 #>=0
    if palN==0:
        print("wire signed [2*IN_WIDTH:0] DPpart{};".format(NumOfMaxSizeComponents), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{};".format(palN, NumOfMaxSizeComponents), file=of)
    print("DotProduct_Systolic_NR_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(NonMaxSizeComponentSize), file=of)
    print("\tDPS{}(clk, reset, enable,".format(NonMaxSizeComponentSize), file=of)
    print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("A{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("B{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\tDPOutReady{}, //not used".format(NumOfMaxSizeComponents), file=of)
    print("\tDPpart{},".format(NumOfMaxSizeComponents), file=of)
    print("\tDPEarlyOutReady{}); //not used\n".format(NumOfMaxSizeComponents), file=of)
elif NonMaxSizeComponentSize==1:
    palN = -1;
    print("wire signed [(2*IN_WIDTH)-1:0] DPpart{};".format(NumOfMaxSizeComponents), file=of)
    print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
    print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH+1),", file=of)
    print(".MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))", file=of)
#     print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH),", file=of)
#     print(".MULT_PIPE_DEPTH(MULT_PIPE_DEPTH+1))", file=of)
    print("M{}(clk, reset, enable,".format(NumOfMaxSizeComponents), file=of)
    print("inReady,", file=of)
    print("A{}, B{}".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("MOR{}, //not used".format(NumOfMaxSizeComponents), file=of)
    print("DPpart{},".format(NumOfMaxSizeComponents), file=of)
    print("eMOR{}); //not used".format(NumOfMaxSizeComponents), file=of)
    print(file=of)
if NonMaxSizeComponentSize>0:
    PalD = palM-palN; #>=0
    #sign extention (if needed)
    if PalD>0:
        if palM==0:
            print("wire signed [(2*IN_WIDTH):0] DPpart{}E = ".format(NumOfMaxSizeComponents), end='', file=of)    
        else:
            print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{}E = ".format(palM, NumOfMaxSizeComponents), end='', file=of)
    #if palM-palN==0:
    #    print("DPpart{};".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
    #else:
        if palN==-1:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH)-1]}}}},DPpart{}}};".format(PalD, NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)                
        elif palN==0:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH)]}}}},DPpart{}}};".format(PalD, NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)    
        else:
            print("{{{{{}{{DPpart{}[(2*IN_WIDTH)+{}]}}}},DPpart{}}};".format(PalD, NumOfMaxSizeComponents, palN, NumOfMaxSizeComponents), file=of)    
        print(file=of)

wire signed [(2*IN_WIDTH)-1:0] DPpart2;
RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH+1))
M2(clk, reset, enable,
inReady,
A2, B2
MOR2, //not used
DPpart2,
eMOR2); //not used

wire signed [(2*IN_WIDTH)+1:0] DPpart2E = {{2{DPpart2[(2*IN_WIDTH)-1]}},DPpart2};



In [40]:
print("AdderTree_{}_A{}#( .IN_WIDTH((2*IN_WIDTH)+{}) )".format(NumOfComponents, AdderSize, lgMAMCS), file=of)
print("\tAT(clk, reset, enable, DPOutReady,", file=of)
print("\t", end='', file=of)
for i in range(NumOfMaxSizeComponents):
    print("DPpart{}, ".format(i), end='', file=of)
if NonMaxSizeComponentSize!=0:
    print("DPpart{}".format(NumOfMaxSizeComponents), end='', file=of)
    if PalD>0:
        print("E", end='', file=of)
    print(",", end='', file=of)
print("\n\toutReady, DP, earlyOutReady);\n", file=of)

AdderTree_3_A3#( .IN_WIDTH((2*IN_WIDTH)+1) )
	AT(clk, reset, enable, DPOutReady,
	DPpart0, DPpart1, DPpart2E,
	outReady, DP, earlyOutReady);



In [36]:
print("endmodule", file=of)

endmodule


In [37]:
if __Print_To_File>0:
    of.close()